In [ ]:
from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.models import Simulation, SimulationCampaign, SimulationResult, SimulationExecution
from entitysdk.common import ProjectContext

In [ ]:
virtual_lab_id = # Replace with your virtual lab ID
project_id = # Replace with your project ID

token = get_token(environment="staging", auth_mode="daf")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(environment="staging", 
                project_context=project_context, 
                token_manager=token)

In [ ]:
from entitysdk.staging.simulation import stage_simulation
from entitysdk.staging.simulation_result import stage_simulation_result

def stage_single_sim_campaign(simulation_campaign_id, output_dir):
    simulation_campaign = client.get_entity(entity_type=SimulationCampaign, entity_id=simulation_campaign_id)
    simulation_id = simulation_campaign.simulations[0].id
    simulation = client.get_entity(entity_type=Simulation, entity_id=simulation_id)
    
    simulation_execution = client.search_entity(
                entity_type=SimulationExecution,
                query={"used__id": simulation.id}
            ).all()[0]
    
    simulation_result_id = simulation_execution.generated[0].id
    simulation_result = client.get_entity(entity_type=SimulationResult, entity_id=simulation_result_id)
    
    simulation_config_path = stage_simulation_result(
        client=client,
        model=simulation_result,
        output_dir=output_dir,
        simulation_config_file=None,
    )

    return simulation_config_path

In [ ]:
simulation_campaign_id = # Replace with your simulation campaign ID
output_dir = "./simulation_result_0"
simulation_config_path = stage_single_sim_campaign(simulation_campaign_id, output_dir=output_dir)

In [ ]:
import bluepysnap
snap_simulation = bluepysnap.Simulation(simulation_config_path)
spikes = snap_simulation.spikes
pop_spikes = spikes['S1nonbarrel_neurons']
print(pop_spikes.get())

In [ ]:
spikes.filter().raster()